In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow.keras.layers as lay

In [92]:
inputs_word = lay.Input(shape=(10000,))
dense_word_1 = lay.Dense(512, activation='relu')(inputs_word)
dropout_word_1 = lay.Dropout(0.5)(dense_word_1)
dense_word_2 = lay.Dense(256, activation='relu')(dropout_word_1)
dropout_word_2 = lay.Dropout(0.5)(dense_word_2)
dense_word_3 = lay.Dense(128, activation='relu')(dropout_word_2)


inputs_img = lay.Input(shape=(128,128,3,))

conv_1 = lay.Conv2D(4, kernel_size=3, strides=2, padding='same')(inputs_img)
batch_1 = lay.BatchNormalization()(conv_1)
acti_1 = lay.Activation('relu')(batch_1)
maxpool_1 = lay.MaxPooling2D(pool_size=3, strides=2, padding='same')(acti_1)
dropout_conv1 = lay.Dropout(0.3)(maxpool_1)

conv_2 = lay.Conv2D(16, kernel_size=3, strides=2, padding='same')(dropout_conv1)
batch_2 = lay.BatchNormalization()(conv_2)
acti_2 = lay.Activation('relu')(batch_2)
maxpool_2 = lay.MaxPooling2D(pool_size=3, strides=2, padding='same')(acti_2)
dropout_conv2 = lay.Dropout(0.3)(maxpool_2)

conv_3 = lay.Conv2D(64, kernel_size=3, strides=2, padding='same')(dropout_conv2)
batch_3 = lay.BatchNormalization()(conv_3)
acti_3 = lay.Activation('relu')(batch_3)
maxpool_3 = lay.MaxPooling2D(pool_size=3, strides=2, padding='same')(acti_3)

flatten = lay.Flatten()(dropout_conv2)
# dense_in_img = lay.Dense(128, activation='softmax')(flatten)
dropout_img_1 = lay.Dropout(0.4)(flatten)

concat = lay.Concatenate()([dense_word_2, dropout_img_1])

dense_1 = lay.Dense(256, activation='relu')(concat)
# dense_2 = lay.Dense(256, activation='relu')(dense_1)

outputs = lay.Dense(128, activation='softmax')(dropout_word_2)

In [81]:
model = tf.keras.models.Model(inputs = [inputs_word, inputs_img], outputs = outputs)

In [93]:
model = tf.keras.models.Model(inputs = inputs_word, outputs = outputs)

In [82]:
opt_adam = tf.keras.optimizers.Adam(learning_rate=0.0005)

In [94]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [7]:
data = pd.read_csv('train.csv')

In [8]:
train_text = data['overview']
train_target = data['cat3']

In [9]:
categori = train_target.unique()

In [10]:
len(categori)

128

In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train_target)
train_target_label = encoder.transform(train_target)

In [12]:
train_img = np.load('test_img.npy')

In [13]:
import rhinoMorph
rn = rhinoMorph.startRhino()

filepath:  C:\Anaconda3\lib\site-packages
classpath:  C:\Anaconda3\lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!


In [14]:
train_morphed = []
for i in tqdm(range(len(train_text))):
    temp = train_text.iloc[i]
    temp_morph = rhinoMorph.onlyMorph_list(rn, temp, pos=['NNG','NNP','VV','VA','XR'])
    train_morphed.append(temp_morph)

100%|███████████████████████████████████████████████████████████████████████████| 16986/16986 [01:04<00:00, 265.17it/s]


In [15]:
with open('한국어불용어100.txt', 'r', encoding='utf-8') as f:
    temp_stopword = f.read().splitlines()
    temp_stopword = list(map(lambda x : x.split('\t'), temp_stopword))

stop_word = []
for i in range(len(temp_stopword)):
    stop_word.append(temp_stopword[i][0])

In [16]:
joined_morph = []
for i in tqdm(range(len(train_morphed))):
    temp = []
    for word in train_morphed[i]:
        if word not in stop_word:
            temp.append(word)
    joined_morph.append(' '.join(temp))
    

100%|█████████████████████████████████████████████████████████████████████████| 16986/16986 [00:01<00:00, 15153.32it/s]


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [95]:
cntvec = CountVectorizer(max_features=10000)

In [96]:
cntvec.fit(joined_morph)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=10000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [97]:
es = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [98]:
train_text_vec = cntvec.transform(joined_morph).toarray()

In [88]:
train_input = [train_text_vec, train_img]

In [99]:
history = model.fit(train_text_vec, train_target_label,epochs=20, batch_size=64, validation_split=0.2, callbacks=[es])

Epoch 1/20
213/213 [==============================] - 5s 24ms/step - loss: 2.4715 - acc: 0.5115 - val_loss: 1.4226 - val_acc: 0.7039
Epoch 2/20
213/213 [==============================] - 5s 23ms/step - loss: 1.1360 - acc: 0.7465 - val_loss: 1.1092 - val_acc: 0.7496
Epoch 3/20
213/213 [==============================] - 5s 23ms/step - loss: 0.6584 - acc: 0.8424 - val_loss: 1.0630 - val_acc: 0.7628
Epoch 4/20
213/213 [==============================] - 5s 23ms/step - loss: 0.3923 - acc: 0.9020 - val_loss: 1.1121 - val_acc: 0.7613
Epoch 5/20
213/213 [==============================] - 5s 23ms/step - loss: 0.2551 - acc: 0.9352 - val_loss: 1.1804 - val_acc: 0.7575
Epoch 6/20
213/213 [==============================] - 5s 24ms/step - loss: 0.1799 - acc: 0.9546 - val_loss: 1.2141 - val_acc: 0.7590
Epoch 7/20
213/213 [==============================] - 5s 24ms/step - loss: 0.1285 - acc: 0.9692 - val_loss: 1.2669 - val_acc: 0.7572
Epoch 8/20
213/213 [==============================] - 5s 24ms/step - 

In [25]:
test_data = pd.read_csv('test.csv')

In [26]:
test_img = []
for i in tqdm(range(len(test_data))):
    temp = Image.open(test_data['img_path'].iloc[i])
    temp = np.array(temp.resize((128,128)))
    test_img.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████| 7280/7280 [01:28<00:00, 81.99it/s]


In [27]:
test_morphed = []
for i in tqdm(range(len(test_data))):
    temp = test_data['overview'].iloc[i]
    temp_morph = rhinoMorph.onlyMorph_list(rn, temp, pos=['NNG','NNP','VV','VA','XR'])
    test_morphed.append(temp_morph)

100%|█████████████████████████████████████████████████████████████████████████████| 7280/7280 [00:28<00:00, 258.53it/s]


In [28]:
test_joined = list(map(lambda x:' '.join(x), test_morphed))

In [29]:
test_morphed_vec = cntvec.transform(test_joined).toarray()

In [30]:
test_input = [test_morphed_vec, np.array(test_img)]

In [100]:
result = model.predict(test_morphed_vec)

In [101]:
result_idx = pd.Series(np.argmax(result, axis=1)).apply(lambda x : encoder.classes_[x])

In [102]:
result_idx

0              한식
1              한식
2              한식
3            수련시설
4               산
          ...    
7275           한식
7276          문화원
7277    야영장,오토캠핑장
7278           모텔
7279          전시관
Length: 7280, dtype: object

In [103]:
np.unique(result_idx, return_counts=True)

(array(['5일장', '강', '게스트하우스', '계곡', '고궁', '고택', '골프', '공연장', '공예,공방', '공원',
        '관광단지', '국립공원', '기념관', '기념탑/기념비/전망대', '기암괴석', '기타', '기타행사',
        '농.산.어촌 체험', '다리/대교', '대형서점', '도서관', '동굴', '동상', '등대', '래프팅', '모텔',
        '문', '문화원', '문화전수시설', '미술관/화랑', '민물낚시', '민박', '민속마을', '바/까페',
        '바다낚시', '박람회', '박물관', '분수', '사찰', '산', '상설시장', '생가', '서양식', '섬',
        '성', '수련시설', '수목원', '스키(보드) 렌탈샵', '승마', '식음료', '썰매장', '안보관광',
        '야영장,오토캠핑장', '약수터', '온천/욕장/스파', '윈드서핑/제트스키', '유람선/잠수함관광', '유명건물',
        '유스호스텔', '유원지', '유적지/사적지', '이색거리', '일반축제', '일식', '자동차경주',
        '자연생태관광지', '자연휴양림', '자전거하이킹', '전문상가', '전시관', '종교성지', '중식', '카트',
        '컨벤션센터', '테마공원', '트래킹', '특산물판매점', '패밀리레스토랑', '펜션', '폭포', '한식',
        '한옥스테이', '항구/포구', '해수욕장', '해안절경', '호수', '홈스테이', '희귀동.식물'],
       dtype=object),
 array([  80,   33,   31,   94,   16,   26,   87,   50,   11,  192,   44,
           1,   39,   66,    9,   17,   27,  143,    5,    3,   44,    4,
           4,    7,    4,  186,    2,   54,   

In [37]:
np.unique(data['cat3'],return_counts=True)

(array(['5일장', 'ATV', 'MTB', '강', '게스트하우스', '계곡', '고궁', '고택', '골프', '공연장',
        '공예,공방', '공원', '관광단지', '국립공원', '군립공원', '기념관', '기념탑/기념비/전망대',
        '기암괴석', '기타', '기타행사', '농.산.어촌 체험', '다리/대교', '대중콘서트', '대형서점',
        '도립공원', '도서관', '동굴', '동상', '등대', '래프팅', '면세점', '모텔', '문', '문화관광축제',
        '문화원', '문화전수시설', '뮤지컬', '미술관/화랑', '민물낚시', '민박', '민속마을', '바/까페',
        '바다낚시', '박람회', '박물관', '발전소', '백화점', '번지점프', '복합 레포츠', '분수', '빙벽등반',
        '사격장', '사찰', '산', '상설시장', '생가', '서비스드레지던스', '서양식', '섬', '성',
        '수련시설', '수목원', '수상레포츠', '수영', '스노쿨링/스킨스쿠버다이빙', '스카이다이빙', '스케이트',
        '스키(보드) 렌탈샵', '스키/스노보드', '승마', '식음료', '썰매장', '안보관광', '야영장,오토캠핑장',
        '약수터', '연극', '영화관', '온천/욕장/스파', '외국문화원', '요트', '윈드서핑/제트스키',
        '유람선/잠수함관광', '유명건물', '유스호스텔', '유원지', '유적지/사적지', '이색거리', '이색찜질방',
        '이색체험', '인라인(실내 인라인 포함)', '일반축제', '일식', '자동차경주', '자연생태관광지',
        '자연휴양림', '자전거하이킹', '전문상가', '전시관', '전통공연', '종교성지', '중식', '채식전문점',
        '카약/카누', '카지노', '카트', '컨벤션', '컨벤션센터', '콘도미니엄', '클래식음악회', 

In [104]:
submit_df = pd.read_csv('sample_submission.csv')
submit_df['cat3'] = result_idx
submit_df.to_csv('submit_1007_1.csv', index=False)

In [53]:
test_data['overview'][2]

'장터설렁탕은 남녀노소 누구나 즐길 수 있는 전통 건강식으로 좋은 재료와 전통 조리방식을 고수해 진한 국물 맛을 선보이는 음식점이다. \n오랜 장터설렁탕의 비법으로 조미료를 사용하지 않아 담백하고, 진하고, 시원한 국물을 맛볼 수 있다. 설렁탕 외에도 다양하고 맛있는 메뉴가 있어 가족들과 함께 외식하기 좋다. 넓은 주차장과 아늑한 실내 인테리어로 가족 단위의 모임을 하기에 좋다.'